# Semantic Search on PDF Documents

##### Note: This example requires a KDB.AI endpoint and API key. Sign up for a free [KDB.AI account](https://kdb.ai/get-started).

This example demonstrates how to use KDB.AI to run semantic search on unstructured text documents. 

Semantic search allows users to perform searches based on the meaning or similarity of the data rather than exact matches. It works by converting the query into a vector representation and then finding similar vectors in the database. This way, even if the query and the data in the database are not identical, the system can identify and retrieve the most relevant results based on their semantic meaning.

### Aim
In this tutorial, we'll walk you through the process of performing semantic search on documents, taking PDFs as example, using KDB.AI as the vector store. We will cover the following topics:

0. Load PDF Data
1. Create Sentence Vector Embeddings
2. Store Embeddings in KDB.AI
3. Search For Similar Sentences To A Target Sentence
4. Delete the KDB.AI Table

---

## 0. Setup

### Install dependencies 

In order to successfully run this sample, note the following steps depending on where you are running this notebook:

-***Run Locally / Private Environment:*** The [Setup](https://github.com/KxSystems/kdbai-samples/blob/main/README.md#setup) steps in the repository's `README.md` will guide you on prerequisites and how to run this with Jupyter.


-***Colab / Hosted Environment:*** Open this notebook in Colab and run through the cells.

### Set Environment Variables

In [ ]:
!pip install kdbai_client pypdf sentence_transformers

In [ ]:
### !!! Only run this cell if running the notebook in Colab
### This downloads research paper pdf into Colab
!mkdir ./data 
!wget -P ./data https://raw.githubusercontent.com/KxSystems/kdbai-samples/main/document_search/data/research_paper.pdf

In [1]:
import os

In [2]:
### ignore tensorflow warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

### Import Packages

In [3]:
# load data
import pypdf
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

In [4]:
# embeddings
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

In [5]:
# vector DB
import os
import kdbai_client as kdbai
from getpass import getpass
import time

### Configure Console

In [6]:
pd.set_option("display.max_colwidth", 300)

### Define Helper Functions

In [7]:
def show_df(df: pd.DataFrame) -> pd.DataFrame:
    print(df.shape)
    return df.head()

## 1. Load PDF Data

### Read Text From PDF Document

We leverage the power of PyPDF2 for PDF processing and `nltk` for advanced natural language processing. The code below extracts content from each page of the PDF and processes it to identify sentences.

The PDF we are using is [this research paper](https://arxiv.org/pdf/2308.05801.pdf) presenting information on the formation of Interstellar Objects in the Milky Way.

In [8]:
# Read PDF file
with open("data/research_paper.pdf", "rb") as pdf_file:
    pdf_pages = pypdf.PdfReader(pdf_file).pages
    page_list = [page.extract_text() for page in pdf_pages]

In [9]:
# Concatenate text from each page
full_pdf_text = "".join(page_list)

### Split The Text Into Sentences

<div class="alert alert-block alert-warning">
    <b>Note: </b>
    Before running the following line of code, please ensure that you have installed the English sentence tokenizer as stated in the `README.md` file in this repository.
</div>

In [10]:
# Split the PDF into sentences
pdf_sentences = sent_tokenize(full_pdf_text)
len(pdf_sentences)

591

In [11]:
pdf_sentences[0]

'Draft version August 14, 2023\nTypeset using L ATEX default style in AASTeX631\nThe Galactic Interstellar Object Population: A Framework for Prediction and Inference\nMatthew J. Hopkins\n ,1Chris Lintott\n ,1Michele T. Bannister\n ,2J.'

## 2. Create Sentence Vector Embeddings 

Next, we use the Sentence Transformers library to create embeddings for our collection of sentences.

### Selecting a Sentence Transformer Model

There are 100+ of different types of Sentence Transformers models available - see [HuggingFace](https://huggingface.co/sentence-transformers) for the full list. The diversity among these primarily stems from variations in their training data. Selecting the ideal model for your needs involves matching the domain and task closely, while also considering the benefits of incorporating larger datasets to enhance scale. 

This tutorial will use the `all-MiniLM-L6-v2` pre-trained model. This embedding model can create sentence and document embeddings that can be used for a wide variety of tasks including semantic search which makes it a good choice for our needs.

In [12]:
model = SentenceTransformer("all-MiniLM-L6-v2")

### Generate Sentence Embeddings Using This Model

We prepare embeddings by applying the sentence transformer model to our sentences to encode them. The we do some transformation to get this into DataFrame which is the format accepted by KDB.AI.

In [13]:
# Create embeddings
embeddings_list = model.encode(np.array(pdf_sentences)).tolist()
embeddings_df = pd.DataFrame({"vectors": embeddings_list, "sentences": pdf_sentences})

In [14]:
show_df(embeddings_df)

(591, 2)


,vectors,sentences
0,"[-0.059804268181324005, -0.09221812337636948, 0.058070000261068344, 0.06884292513132095, -0.00304530281573534, 0.0073042758740484715, -0.02895909734070301, 0.05957881733775139, 0.017943765968084335, 0.0435011088848114, 0.005012931302189827, -0.07875000685453415, -0.02570343017578125, -0.04147786...","Draft version August 14, 2023\nTypeset using L ATEX default style in AASTeX631\nThe Galactic Interstellar Object Population: A Framework for Prediction and Inference\nMatthew J. Hopkins\n ,1Chris Lintott\n ,1Michele T. Bannister\n ,2J."
1,"[-0.08154534548521042, -0.1134231761097908, 0.08425557613372803, 0.08849424868822098, 0.024723690003156662, -0.08773959428071976, -0.06128990650177002, 0.01061215903609991, 0.06387816369533539, 0.021902453154325485, -0.03322550654411316, -0.09321600943803787, -0.04425088316202164, -0.05684755370...","Ted Mackereth\n ,3, 4, 5, ∗and\nJohn C. Forbes\n2\n1Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK\n2School of Physical and Chemical Sciences—Te Kura Mat¯ u, University of Canterbury, Private Bag 4800, Christchurch 8140, New Zealand\n3Just ..."
2,"[-0.07799113541841507, -0.08398672193288803, 0.027237098664045334, 0.09858059883117676, 0.010515064932405949, -0.016703935340046883, -0.05579749494791031, 0.027984393760561943, 0.03926646336913109, 0.03248242288827896, 0.038379862904548645, -0.12464363873004913, -0.012645186856389046, -0.0182884...","We define a novel framework: firstly to predict\nthe properties of this Galactic ISO population by combining models of processes across planetary\nand galactic scales, and secondly to make inferences about the processes modelled, by comparing the\npredicted population to what is observed."
3,"[-0.05338568240404129, -0.03714732825756073, 0.07784586399793625, 0.051659490913152695, -0.07674476504325867, -0.007766887079924345, 0.008464711718261242, 0.04289114475250244, 0.04356802999973297, 0.06012841686606407, 0.044070351868867874, -0.10177759826183319, 0.030359644442796707, -0.034037191...",We predict the spatial and compositional distribution of the\nGalaxy’s population of ISOs by modelling the Galactic stellar population with data from the APOGEE\nsurvey and combining this with a protoplanetary disk chemistry model.
4,"[-0.06326980888843536, -0.01720455102622509, 0.04773722216486931, 0.0832536369562149, 0.0329662561416626, -0.0660020187497139, 0.0015915802214294672, 0.0032460016664117575, 0.03456268459558487, -0.020524034276604652, 0.0412253700196743, -0.15274155139923096, -0.011771155521273613, 0.073143251240...","Selecting ISO water mass\nfraction as an example observable quantity, we evaluate its distribution both at the position of the Sun\nand averaged over the Galactic disk; our prediction for the Solar neighbourhood is compatible with the\ninferred water mass fraction of 2I/Borisov."


It is important to note the dimension of our embeddings is 384. This will need to match the dimensions we set in the KDB.AI index in the next step. We can easily check this using `len` to count elements in our vector.

In [15]:
len(embeddings_df["vectors"][0])

384

## 3. Store Embeddings in KDB.AI

With the embeddings created, we need to store them in a vector database to enable efficient searching.

### Define KDB.AI Session

KDB.AI comes in two offerings:

1. [KDB.AI Cloud](https://trykdb.kx.com/kdbai/signup/) - For experimenting with smaller generative AI projects with a vector database in our cloud.
2. [KDB.AI Server](https://trykdb.kx.com/kdbaiserver/signup/) - For evaluating large scale generative AI applications on-premises or on your own cloud provider.

Depending on which you use there will be different setup steps and connection details required.

##### Option 1. KDB.AI Cloud

To use KDB.AI Cloud, you will need two session details - a URL endpoint and an API key.
To get these you can sign up for free [here](https://trykdb.kx.com/kdbai/signup).

You can connect to a KDB.AI Cloud session using `kdbai.Session` and passing the session URL endpoint and API key details from your KDB.AI Cloud portal.

If the environment variables `KDBAI_ENDPOINTS` and `KDBAI_API_KEY` exist on your system containing your KDB.AI Cloud portal details, these variables will automatically be used to connect.
If these do not exist, it will prompt you to enter your KDB.AI Cloud portal session URL endpoint and API key details.

In [16]:
KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
    if "KDBAI_ENDPOINT" in os.environ
    else input("KDB.AI endpoint: ")
)
KDBAI_API_KEY = (
    os.environ["KDBAI_API_KEY"]
    if "KDBAI_API_KEY" in os.environ
    else getpass("KDB.AI API key: ")
)

In [17]:
session = kdbai.Session(api_key=KDBAI_API_KEY, endpoint=KDBAI_ENDPOINT)

##### Option 2. KDB.AI Server

To use KDB.AI Server, you will need download and run your own container.
To do this, you will first need to sign up for free [here](https://trykdb.kx.com/kdbaiserver/signup/). 

You will receive an email with the required license file and bearer token needed to download your instance.
Follow instructions in the signup email to get your session up and running.

Once the [setup steps](https://code.kx.com/kdbai/gettingStarted/kdb-ai-server-setup.html) are complete you can then connect to your KDB.AI Server session using `kdbai.Session` and passing your local endpoint.

In [ ]:
# session = kdbai.Session(endpoint="http://localhost:8082")

### Define Vector DB Table Schema

The next step is to define a schema for our KDB.AI table where we will store our embeddings. Our table will have two columns.

At this point you will select the index and metric you want to use for searching.

With KDB.AI we have the choice between HNSW (Hierarchical Navigable Small World), IVF, IVFPQ and Flat indexing methods. Generally, for semantic search of documents, the HNSW indexing method might be more suitable. Here's why:

- **Search Speed and Approximation**: HNSW is designed for fast approximate nearest neighbor searches. It can efficiently handle high-dimensional data, which is common in natural language processing tasks involving text documents.
- **Semantic Representation**: The Sentence Transformers library, used in this example, generates embeddings that capture semantic meaning. HNSW is well-suited for indexing such embeddings and performing semantic searches.
- **Scalability**: HNSW is scalable and can handle large datasets effectively, making it suitable for applications with a vast number of documents.

HNSW provides approximate search results, meaning that the nearest neighbors might not be exact matches but are close in terms of similarity.

In [18]:
pdf_schema = {
    "columns": [
        {"name": "sentences", "pytype": "str"},
        {
            "name": "vectors",
            "vectorIndex": {"dims": 384, "metric": "L2", "type": "hnsw"},
        },
    ]
}

### Create Vector DB Table

Use the KDB.AI `create_table` function to create a table that matches the defined schema in the vector database.

In [19]:
# First ensure the table does not already exist
try:
    session.table("pdf").drop()
    time.sleep(5)
except kdbai.KDBAIException:
    pass

In [20]:
table = session.create_table("pdf", pdf_schema)

We can use `query` to see our table exists but is empty.

In [21]:
table.query()

,sentences,vectors


### Add Embedded Data to KDB.AI Table

In [22]:
table.insert(embeddings_df)

True

### Verify Data Has Been Inserted

Running `table.query()` should show us that data has been added.

In [23]:
show_df(table.query())

(591, 2)


,sentences,vectors
0,"Draft version August 14, 2023\nTypeset using L ATEX default style in AASTeX631\nThe Galactic Interstellar Object Population: A Framework for Prediction and Inference\nMatthew J. Hopkins\n ,1Chris Lintott\n ,1Michele T. Bannister\n ,2J.","[-0.059804268181324005, -0.09221812337636948, 0.058070000261068344, 0.06884292513132095, -0.00304530281573534, 0.0073042758740484715, -0.02895909734070301, 0.05957881733775139, 0.017943765968084335, 0.0435011088848114, 0.005012931302189827, -0.07875000685453415, -0.02570343017578125, -0.04147786..."
1,"Ted Mackereth\n ,3, 4, 5, ∗and\nJohn C. Forbes\n2\n1Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK\n2School of Physical and Chemical Sciences—Te Kura Mat¯ u, University of Canterbury, Private Bag 4800, Christchurch 8140, New Zealand\n3Just ...","[-0.08154534548521042, -0.1134231761097908, 0.08425557613372803, 0.08849424868822098, 0.024723690003156662, -0.08773959428071976, -0.06128990650177002, 0.01061215903609991, 0.06387816369533539, 0.021902453154325485, -0.03322550654411316, -0.09321600943803787, -0.04425088316202164, -0.05684755370..."
2,"We define a novel framework: firstly to predict\nthe properties of this Galactic ISO population by combining models of processes across planetary\nand galactic scales, and secondly to make inferences about the processes modelled, by comparing the\npredicted population to what is observed.","[-0.07799113541841507, -0.08398672193288803, 0.027237098664045334, 0.09858059883117676, 0.010515064932405949, -0.016703935340046883, -0.05579749494791031, 0.027984393760561943, 0.03926646336913109, 0.03248242288827896, 0.038379862904548645, -0.12464363873004913, -0.012645186856389046, -0.0182884..."
3,We predict the spatial and compositional distribution of the\nGalaxy’s population of ISOs by modelling the Galactic stellar population with data from the APOGEE\nsurvey and combining this with a protoplanetary disk chemistry model.,"[-0.05338568240404129, -0.03714732825756073, 0.07784586399793625, 0.051659490913152695, -0.07674476504325867, -0.007766887079924345, 0.008464711718261242, 0.04289114475250244, 0.04356802999973297, 0.06012841686606407, 0.044070351868867874, -0.10177759826183319, 0.030359644442796707, -0.034037191..."
4,"Selecting ISO water mass\nfraction as an example observable quantity, we evaluate its distribution both at the position of the Sun\nand averaged over the Galactic disk; our prediction for the Solar neighbourhood is compatible with the\ninferred water mass fraction of 2I/Borisov.","[-0.06326980888843536, -0.01720455102622509, 0.04773722216486931, 0.0832536369562149, 0.0329662561416626, -0.0660020187497139, 0.0015915802214294672, 0.0032460016664117575, 0.03456268459558487, -0.020524034276604652, 0.0412253700196743, -0.15274155139923096, -0.011771155521273613, 0.073143251240..."


## 4. Search For Similar Sentences To A Target Sentence

Now that the embeddings are stored in KDB.AI, we can perform semantic search using `search`. 

### Search 1

First, we embed our search term using the Sentence Transformer model as before. Then we search our index to return the three most similar vectors.

In [24]:
search_term1 = "number of interstellar objects in the milky way"

In [25]:
encoded_search_term1 = model.encode(search_term1).tolist()

In [26]:
results1 = table.search([encoded_search_term1], n=3)
results1[0]

,sentences,vectors,__nn_distance
0,"Ted Mackereth\n ,3, 4, 5, ∗and\nJohn C. Forbes\n2\n1Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK\n2School of Physical and Chemical Sciences—Te Kura Mat¯ u, University of Canterbury, Private Bag 4800, Christchurch 8140, New Zealand\n3Just ...","[-0.08154534548521042, -0.1134231761097908, 0.08425557613372803, 0.08849424868822098, 0.024723690003156662, -0.08773959428071976, -0.06128990650177002, 0.01061215903609991, 0.06387816369533539, 0.021902453154325485, -0.03322550654411316, -0.09321600943803787, -0.04425088316202164, -0.05684755370...",0.642638
1,"In this work, we develop\nthis method and apply it to the stellar population of the Milky Way, estimated with data from the APOGEE survey, to\npredict a broader set of properties of our own Galaxy’s population of interstellar objects.","[-0.08688255399465561, -0.023635543882846832, 0.06744849681854248, 0.08018781244754791, -0.05974710360169411, -0.032227396965026855, -0.008770059794187546, 0.04886196553707123, -0.05181039124727249, 0.01759468950331211, 0.024308934807777405, -0.09147436916828156, 0.03819924220442772, -0.06181903...",0.668817
2,"Keywords: Interstellar objects (52), Small Solar System bodies(1469), Galaxy Evolution (594)\n1.INTRODUCTION\n1I/‘Oumuamua (Meech et al.","[-0.07292623072862625, 0.02145012468099594, 0.01660236530005932, 0.051251962780952454, -0.017564918845891953, -0.08743888139724731, -0.02064172923564911, 0.03719828650355339, -0.024873126298189163, 0.039150986820459366, 0.04321705549955368, -0.13203547894954681, -0.03365805745124817, -0.06451229...",0.741215


The results returned from `table.search` show the closest matches along with value of nearest neighbor distances `nn_distance`.
We can see these sentences do reference our search term 'number of interstellar objects in the milky way' in some way.

### Search 2

Let's try another search term.

In [27]:
search_term2 = "how does planet formation occur"

In [28]:
encoded_search_term2 = model.encode(search_term2).tolist()

In [29]:
results2 = table.search([encoded_search_term2], n=3)
results2[0]

,sentences,vectors,__nn_distance
0,"The pop-\nulation’s dominant dynamical formation mechanisms would preferentially harvest more distant, ice-rich planetesimals\nfrom the disks of the source systems.","[-0.07665152102708817, -0.06582032889127731, 0.03430570662021637, 0.026705782860517502, 0.07752171158790588, -0.05098922923207283, 0.007996325381100178, 0.023464038968086243, 0.09635236859321594, 0.05890347808599472, -0.00934863742440939, -0.04947571083903313, 0.04072216525673866, -0.08664802461...",0.907237
1,"A protoplanetary disk has to first order the same composition as the star it forms around,\nsince they both form from the same molecular cloud core.","[-0.011731932871043682, -0.0626753717660904, 0.08392807096242905, -0.036336500197649, -0.0021243898663669825, -0.05082395672798157, -0.00048589648213237524, -0.02759409509599209, 0.1368178427219391, 0.06662701815366745, -0.02651246450841427, -0.01938665471971035, 0.016084985807538033, -0.0834580...",0.932817
2,"While in reality, stars will each produce a distribution of ISOs that\nformed at different positions in their protoplanetary disk and thus have a range of compositions, this simplification\nof only modelling planetesimals which form exterior to the water ice line is justified by the proportional...","[-0.0878973975777626, -0.053996723145246506, 0.08170220255851746, 0.05057612806558609, -0.0012670933501794934, -0.05180908739566803, -0.036422982811927795, 0.014396519400179386, 0.10822281986474991, 0.04368608072400093, -0.07996443659067154, -0.07062524557113647, 0.05373862758278847, -0.06110461...",0.954052


Again, we can see these sentences do reference our search term 'how does planet formation occur' in some way. 

## 5. Delete the KDB.AI Table

Once finished with the table, it is best practice to drop it.

In [30]:
table.drop()

True

## Take Our Survey

We hope you found this sample helpful! Your feedback is important to us, and we would appreciate it if you could take a moment to fill out our brief survey. Your input helps us improve our content.

[**Take the Survey**](https://delighted.com/t/ezUwZ88a)